# Supervised models predicting cubic perovskites

In this notebook, we will indulge into different unsupervised models. So far, this includes RandomForest and Gradient Boost. We will be using the library of scikitlearn to do this.

The notebook has been modular for further implementation of other machine learning algorithms. It should be relatively straight forward to add more algorithms. 

Starting off with some basic imports. 

In [1]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.models import train_model, predict_model
from src.features import build_features
from src.visualization import visualize

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#Models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

#Feature selections
from sklearn.model_selection import RepeatedStratifiedKFold

#metrics and nice visualization
from tqdm import tqdm

# setting random seed for reproducibility
random_state=98127480

from pathlib import Path
data_dir   = Path.cwd().parent / "data" 
models_dir = Path.cwd().parent / "models"
print("Current data directory {}".format(data_dir))
print("Current models directory {}".format(models_dir))

Current data directory /Users/ohebbi/Documents/UiO/Masterprosjekt/predicting-perovskites-v2/data


Then we can read the preprocessed data using pandas library. 

In [4]:
X = pd.read_csv(data_dir / "processed" / "X.csv")
y = pd.read_csv(data_dir / "processed" / "target.csv")
data = pd.read_csv(data_dir / "processed" / "data.csv")

X

,rA,rB,MA,MB,dAO,dBO,rA/rO,rB/rO,t
0,1.460,0.760,65,86,1.805,2.060,1.081,0.563,0.942
1,1.460,0.470,65,95,1.805,1.840,1.081,0.348,1.092
2,1.460,0.130,65,82,1.805,1.432,1.081,0.096,1.343
3,1.460,0.380,65,83,1.805,1.604,1.081,0.281,1.149
4,1.460,0.600,65,85,1.805,1.942,1.081,0.444,1.019
...,...,...,...,...,...,...,...,...,...
385,1.196,0.645,12,52,2.014,1.732,0.886,0.478,0.902
386,1.196,0.600,12,61,2.014,1.750,0.886,0.444,0.923
387,1.196,0.745,12,11,2.014,1.849,0.886,0.552,0.859
388,1.196,0.670,12,43,2.014,1.791,0.886,0.496,0.891


# Algorithms

Below we define the algorithm to use and its abbreviation. Parameters that are optional to tune are the parameters to the algoriths, with the default value as their optimised value. Another parameter to tune is how many cross-validations one wants to iterate through for the analysis. In addition, one has to find the best features for a new algorithm which will be added further down in the notebook.

In [5]:
InsertAlgorithms    = [RandomForestClassifier    (),\
                       GradientBoostingClassifier()]
InsertAbbreviations = ["RF", "GB"]
InsertPrettyNames   = ["Random Forest", "Gradient Boost"]

# Dataset analysis

## Optimal hyperparameters search

In this section we will find the optimal parameters used for the various algorithms. We will use imblearn's Pipeline and its implemented samplers, such as SMOTE and RandomUnderSampler. The advantage of using imblearn instead of sklearn, is that sklearn's pipeline will fit the samplers to the validation data as well, while imblearn only fit the resamplers to the training data.

In [6]:
## TEST

#rskfold = RepeatedStratifiedKFold(n_splits=numberSplits, n_repeats=numberRuns, random_state=random_state)

#a, grid = applyGridSearch(X = X, y = y.values.reshape(-1,), model=InsertAlgorithms[0], cv = rskfold, sampleMethod="")

In [7]:
numberRuns=10
numberSplits = 10

includeSampleMethods = ["", "under", "over", "both"]

Abbreviations = []
prettyNames   = []
Algorithms = []

rskfold = RepeatedStratifiedKFold(n_splits=numberSplits, n_repeats=numberRuns, random_state=random_state)

PerovskiteModelsBestParams = pd.Series({}, dtype="string")

for i, algorithm in tqdm(enumerate(InsertAlgorithms)):
    for method in includeSampleMethods:
        print("Finding best params for: {}".format(InsertAbbreviations[i] + " " + method))
        bestEstimator, PerovskiteModelsBestParams[InsertAbbreviations[i] + " " + method] = train_model.applyGridSearch(
                                                                            X = X, 
                                                                            y = y.values.reshape(-1,), 
                                                                        model = algorithm, 
                                                                           cv = rskfold, 
                                                                 sampleMethod = method)
        Abbreviations.append(InsertAbbreviations[i] + " " + method)
        prettyNames.append(InsertAbbreviations[i] + " " + method)
        Algorithms.append(bestEstimator)

0it [00:00, ?it/s]

Finding best params for: RF 


0it [03:42, ?it/s]


KeyboardInterrupt: 

### One time cross-validating 100 for learning perovskite structure
Under follows the general model runModel that takes the as parameter which model to run and returns nice statistics formatted as a dictionary. 

## Running the different supervised models
This is where we generate a lot of different models, and thus will take some time to execute. 



In [ ]:
PerovskiteModels = pd.Series({}, dtype="string")

In [ ]:
for i, algorithm in enumerate(Algorithms): 
    print("Current training algorithm: {}".format(prettyNames[i]))
    PerovskiteModels[Abbreviations[i]] = (
        train_model.runSupervisedModel(classifier  = algorithm, 
                                     X = X,
                                     y = y.values.reshape(-1,),
                                     k = numberSplits,
                                     n = numberRuns,
                      resamplingMethod = "under",
                     featureImportance = True))

### Visualising the results

In [ ]:
visualize.plot_accuracy(PerovskiteModels, prettyNames)

The standard deviation is calculated as a function difference of the 100 models in the purpose of visalizing how much the models deviate from each other.
### Interpreting important features used by the model

In [ ]:
visualize.plot_important_features(PerovskiteModels, prettyNames, X=X, k = numberSplits, n = numberRuns)

Here we see that random forrest is heavily dependent on four features, while gradient boost is more dependent on three features. We set a limit that the feature needs to be important in at least 50 percent of the classifications.

Feature importance is here calculated from the last iteration of the training, and not as mean from the entire dataset. 

## "Confusion" metrics

### Sorting as a function of alphabetic order

In [ ]:
visualize.plot_confusion_metrics(PerovskiteModels, prettyNames, data, k = numberSplits, n = numberRuns)

In these plots we see that gradient boost is more likely to misclassify a wider variation of compounds, with an exception for MgSiO3 for false negatives. Random forest, on the other hand, is more likely to misclassify the same compounds for all models.  
### Sorting as a function of tolerance factor

In [ ]:
fig = go.Figure( 
    layout = go.Layout (
        title=go.layout.Title(text="False positives (Nruns = {})".format(numberSplits*numberRuns)),
        yaxis=dict(title='Counts'),
        xaxis=dict(title='tolerance factor')
        )
    )

for i, model in enumerate(PerovskiteModels):
    fig.add_traces(go.Bar(name=prettyNames[i], 
                          x=data['t'][model['falsePositives'] > 0],
                          y=model['falsePositives'][model['falsePositives'] > 0],
                          text=data['Compound'][model['falsePositives'] > 0],
                          )
                  )

fig.update_layout(barmode='group')
fig.add_trace(go.Scatter(x=[0.73,0.75], y=[5,5], fill='tozeroy',
                    mode= 'none', name='Not a perovskite'))

fig.add_trace(go.Scatter(x=[0.75,0.9], y=[10,10], fill='tozeroy',
                    mode= 'none', name='Orthorombic perovskite'))
fig.add_trace(go.Scatter(x=[0.9,1.0], y=[10,10], fill='tozeroy',
                    mode= 'none', name='Cubic perovskite'))
fig.add_trace(go.Scatter(x=[1.0,1.15], y=[5,5], fill='tozeroy',
                    mode= 'none', name='Hexagonal nonperovskite'))

fig.show()

From the false positive plot above, we see that there are several compounds with the same tolerance factor, giving rise to several more counts per tolerance factor compared to what is actually shown.

In [ ]:
fig = go.Figure( 
    layout = go.Layout (
        title=go.layout.Title(text="falseNegatives (Nruns = {})".format(numberSplits*numberRuns)),
        yaxis=dict(title='Counts'),
        xaxis=dict(title='tolerance factor')
        )
    )

for i, model in enumerate(PerovskiteModels):
    fig.add_traces(go.Bar(name=prettyNames[i], 
                          x=data['t'][model['falseNegatives'] > 0],
                          y=model['falseNegatives'][model['falseNegatives'] > 0],
                          text=data['Compound'][model['falseNegatives'] > 0],
                          )
                  ) 
fig.update_layout(barmode='group')

fig.add_trace(go.Scatter(x=[0.73,0.75], y=[5,5], fill='tozeroy',
                    mode= 'none', name='Not a perovskite'))

fig.add_trace(go.Scatter(x=[0.75,0.9], y=[10,10], fill='tozeroy',
                    mode= 'none', name='Orthorombic perovskite'))
fig.add_trace(go.Scatter(x=[0.9,1.0], y=[10,10], fill='tozeroy',
                    mode= 'none', name='Cubic perovskite'))
fig.add_trace(go.Scatter(x=[1.0,1.15], y=[5,5], fill='tozeroy',
                    mode= 'none', name='Hexagonal nonperovskite'))

fig.show()

In [ ]:
fig = go.Figure( 
    layout = go.Layout (
        title=go.layout.Title(text="falseNegatives (Nruns = {})".format(numberRuns*numberSplits)),
        yaxis=dict(title='Counts'),
        xaxis=dict(title='MA')
        )
    )

for i, model in enumerate(PerovskiteModels):
    fig.add_traces(go.Bar(name=prettyNames[i], 
                          x=data['MA'][model['falseNegatives'] > 0],
                          y=model['falseNegatives'][model['falseNegatives'] > 0],
                          text=data['Compound'][model['falseNegatives'] > 0],
                          )
                  ) 
fig.show()

In [ ]:
fig = go.Figure( 
    layout = go.Layout (
        title=go.layout.Title(text="falseNegatives (Nruns = {})".format(numberRuns*numberSplits)),
        yaxis=dict(title='Counts'),
        xaxis=dict(title='MB')
        )
    )

for i, model in enumerate(PerovskiteModels):
    fig.add_traces(go.Bar(name=prettyNames[i], 
                          x=data['MB'][model['falseNegatives'] > 0],
                          y=model['falseNegatives'][model['falseNegatives'] > 0],
                          text=data['Compound'][model['falseNegatives'] > 0],
                          )
                  ) 
fig.show()

# Which perovskites are correctly predicted?

Out of 100 different runs, the article yield a prediction if over 50% of the predictions are in favor for either perovskite or not. 

In [ ]:
visualize.plot_confusion_matrix(PerovskiteModels, y, data, abbreviations=Abbreviations, names=prettyNames, k = numberSplits, n = numberRuns)

From the plot above, we can choose whichever confidence we would like to make a confusion matrix. The article has used a binomial distribution, without doing the statistics behind a one-side hypothesis test. Let us do better. 

We will start by excluding every compound that has been misclassified over 50% of the classifications.

In [ ]:
visualize.confusion_matrix_plot(PerovskiteModels, y, prettyNames)

### Preparing training data for the cubic classification

After training a model on the perovskite data, we would like to make a smaller subset of the dataset as the training data for a model that can classify a cubic perovskite versus a non-cubic perovskite. We will remove all compounds that got misclassified at least 50% of 100 classifications, in addition to non-perovskites. Thus, if two algorithms is seen with same compounds that are being removed due to false negatives but different number of false positives, they will have the same test data.

To start of, which compounds are predicted most times as perovskite, but are in fact non-perovskite? 

In [ ]:
for i, model in enumerate(PerovskiteModels):
    print(prettyNames[i], ": over 50 percent misclassifications:")
    print("False negatives:")
    print(data['Compound'][model['falseNegatives'] > numberRuns*numberSplits/2])
    print("False positives:")
    print(data['Compound'][model['falsePositives'] > numberRuns*numberSplits/2])
    print("")

In [ ]:
allowedOfNumberMiscalculations = 50
correctlyPredictedPerovskites = {}

for abbreviation in Abbreviations:     
    correctlyPredictedPerovskites[abbreviation] = visualize.findCorrectlyPredictedPerovskites(
        PerovskiteModels[abbreviation], data, allowedOfNumberMiscalculations)

correctlyPredictedPerovskites["GB"]

## Save the models 

Now, we will save the perovskite-models for reproducability.

In [ ]:
import pickle
for i, algorithm in tqdm(enumerate(Algorithms)):
    file_path = Path(models_dir / "perovskite" / Path(prettyNames[i] + ".pkl"))
    with file_path.open('wb') as fp:
        pickle.dump(algorithm, fp)

In [ ]:
correctlyPredictedPerovskites["RF "]

# Training the model for cubic perovskite or not

## Finding the optimal hyperparameters imblearn

We will here do the same procedure as for the perovskite model, however, with the added complexity that the different models will run on different data depending on what the model predicted for perovskites. 

In [ ]:
rskfold = RepeatedStratifiedKFold(n_splits=numberSplits, n_repeats=numberRuns, random_state=random_state)
allFeatures = ["rA", "rB", "MA", "MB", "dAO", "dBO", "rA/rO", "rB/rO", "t"]
CubicModelsBestParams = pd.Series({}, dtype="string")
BestParamsAlgorithms = []
for i, algorithm in tqdm(enumerate(InsertAlgorithms)):
    for method in includeSampleMethods:
        print("Finding best params for: {}".format(InsertAbbreviations[i] + " " + method))
        bestEstimator, CubicModelsBestParams[InsertAbbreviations[i] + " " + method] = = train_model.applyGridSearch(
                                                                            X = correctlyPredictedPerovskites[InsertAbbreviations[i] + " " + method][allFeatures],
                                                                            y = correctlyPredictedPerovskites[InsertAbbreviations[i] + " " + method]["Cubic"].values.reshape(-1,),
                                                                        model = algorithm, 
                                                                           cv = rskfold, 
                                                                 sampleMethod = method)
        # Only the parameters for the algorithm changes here, 
        # since pretty name and abbreviations are already done.
        BestParamsAlgorithms.append(bestEstimator)

In [ ]:
CubicModels = pd.Series({}, dtype="string")
for i, algorithm in enumerate(BestParamsAlgorithms): 
    print("Current training algorithm: {}".format(prettyNames[i]))
    CubicModels[Abbreviations[i]] = (
        train_model.runSupervisedModel(classifier  = algorithm, 
                                     X = correctlyPredictedPerovskites[Abbreviations[i]][allFeatures],
                                     y = correctlyPredictedPerovskites[Abbreviations[i]]["Cubic"].values.reshape(-1,),
                                     k = numberSplits,
                                     n = numberRuns,
                      resamplingMethod = "None",
                     featureImportance = True))

In [ ]:
visualize.plot_accuracy(CubicModels, prettyNames)

In [ ]:
visualize.plot_important_features(CubicModels, prettyNames, X, k = numberSplits, n = numberRuns)

In [ ]:
for i, model in enumerate(CubicModels):
    print(X.columns[model["importantKeys"]>0])

Once again, we see here that some of the features are redundant, as both random forrest and gradient boost agrees on rA, rA/rO and t as features. 

In [ ]:
visualize.plot_confusion_metrics(CubicModels, 
                                 prettyNames, 
                                 correctlyPredictedPerovskites, 
                                 k = numberSplits, 
                                 n = numberRuns, 
                                 abbreviations = Abbreviations, 
                                 cubicCase = True)

In [ ]:
visualize.plot_confusion_matrix(CubicModels,\
                      correctlyPredictedPerovskites,\
                      correctlyPredictedPerovskites,
                      abbreviations = Abbreviations,
                      names = prettyNames,
                      k = numberSplits,
                      n = numberRuns,
                      cubicCase = True
                               )

# Predictions on the test set

We will be using the generated training datasets for both the perovskite and cubic classification to train a model, which will be used to classify a completely independent test dataset. 

## Predicting perovskites

In [ ]:
PerovskiteTestData = pd.read_csv(data_dir / "processed" / "625TestData.csv")

Summary             = pd.DataFrame({}, dtype="string")
Summary["Compound"] = PerovskiteTestData["Compound"]

PredictedPerovskites = pd.Series({}, dtype="string")

threshold = numberSplits*numberRuns/2 #50% when equal

for i, algorithm in enumerate(Algorithms):#
    PredictedPerovskites[Abbreviations[i]+" P"], PredictedPerovskites[Abbreviations[i]+" P Prob"] = predict_model.runPredictions(algorithm,\
                                    trainingData   = X[X.columns[PerovskiteModels[Abbreviations[i]]["importantKeys"]>threshold]],\
                                    trainingTarget = y.values.reshape(-1,),\
                                    testData   = PerovskiteTestData[PerovskiteTestData[allFeatures].columns[PerovskiteModels[Abbreviations[i]]["importantKeys"]>threshold]])

for abbreviation in Abbreviations:
    PerovskiteTestData[abbreviation + " P"] = PredictedPerovskites[abbreviation + " P"]
    Summary[abbreviation + " P"]            = PredictedPerovskites[abbreviation + " P"]
    Summary[abbreviation + " P Prob"]       = PredictedPerovskites[abbreviation + " P Prob"]
    print(abbreviation, "predict the number of perovskites as: ", np.sum(PredictedPerovskites[abbreviation+" P"]))

In [ ]:
Summary

## Finding training and test data for the cubic prediction
Now, we need to remove all the non-perovskite in order to classify them into cubic-perovskites or non-cubic perovskites. Alas, the results above does imply many different training sets because of different results from the classifiers from the previous classification.  

In [ ]:
CubicTestData = pd.Series({}, dtype="string")

def RemoveNonPerovskites(predictions):
    remove_indices = np.array(PerovskiteTestData['Compound'][predictions == 0].index)
    cubicTestData = PerovskiteTestData.drop(index=remove_indices)
    cubicTestData.reset_index(drop=False, inplace=True)
    return cubicTestData

for abbreviation in Abbreviations:
    CubicTestData[abbreviation] = RemoveNonPerovskites(PerovskiteTestData[abbreviation+" P"])

In [ ]:
print(CubicTestData["RF "].shape)
print(CubicTestData["GB "].shape)

However, here we meet a challenge since the training data does only contain less than 9 percent cubic perovskites. When training the model earlier, we used a stratified cross fold validation scheme to create training and validation sets on the basis of a 50/50 split. We did this by the help of a stratified parameter which is built into SciKitLearn. Now, we would like to use all of the cubic data while keeping it 50/50 on the entire dataset, but this we need to make ourself. 

In [ ]:
#correctlyPredictedPerovskites

In [ ]:
#Not using pandas dataframe because the contents does not neccessarily have the same shape
stratifiedCubicData = pd.Series({}, dtype="string")

#choose percentage of data that should be cubic. 50/50 split is percentage=0.085
percentage = 0.25

def getStratifiedTrainingData(predictedPerovskites, Name):
    cubics    = predictedPerovskites.iloc[predictedPerovskites["Cubic"][predictedPerovskites["Cubic"]==1].index]
    nonCubics = predictedPerovskites.iloc[predictedPerovskites["Cubic"][predictedPerovskites["Cubic"]!=1].index]
    print(Name, ":")
    print("The amount of cubic perovskites entries in the data is {}, with a total percentage of {}"\
          .format(np.sum(cubics["Cubic"]), np.sum(cubics["Cubic"])/len(predictedPerovskites["Cubic"])))
    
    # The data trained on should be evenly distributed. Here, we are just picking random numbers. 
    nonCubicsSubSet = nonCubics.sample(n = int(percentage*len(predictedPerovskites.index)), random_state=random_state)

    #test to make the reader aware of the distribution in the training data
    if (nonCubicsSubSet.shape!=cubics.shape):
        print("Current shape Cubics: {} and nonCubics: {}".format(cubics.shape, nonCubicsSubSet.shape))
    
    #Combining the subsets
    stratCubicData = pd.concat([cubics, nonCubicsSubSet])
    stratCubicData.reset_index(drop=False, inplace=True)
    return stratCubicData

for abbreviation in Abbreviations: 
    print("Name  :{}".format(abbreviation))
    print("Shape :{}".format(correctlyPredictedPerovskites[abbreviation].shape))
    stratifiedCubicData[abbreviation] = getStratifiedTrainingData(correctlyPredictedPerovskites[abbreviation], abbreviation)

# Predicting cubic perovskites
Now that we have stratified data, we can finally fit our classifier.


In [ ]:
for i, data in enumerate(stratifiedCubicData):
    print("The shape of {} is: {}".format(prettyNames[i], data.shape))

In [ ]:
PredictedCubics = {}
for i, algorithm in enumerate(Algorithms):
    PredictedCubics[Abbreviations[i]+" C"], PredictedCubics[Abbreviations[i]+" C Prob"] = predict_model.runPredictions(algorithm,\
                                    trainingData   = stratifiedCubicData[Abbreviations[i]][stratifiedCubicData[Abbreviations[i]][allFeatures].columns[CubicModels[Abbreviations[i]]["importantKeys"]>threshold]],\
                                    trainingTarget = stratifiedCubicData[Abbreviations[i]]["Cubic"],\
                                    testData   = CubicTestData[Abbreviations[i]][CubicTestData[Abbreviations[i]][allFeatures].columns[CubicModels[Abbreviations[i]]["importantKeys"]>threshold]])
    print(Abbreviations[i], " predict the number of cubic perovskites as: ", np.sum(PredictedCubics[Abbreviations[i]+" C"]))      

In [ ]:
for abbreviation in Abbreviations:
    tmp1 = np.empty(len(Summary.index))
    tmp2 = np.copy(tmp1)
    tmp2[:] = np.nan
    tmp1[:] = -1 #For all non-perovskites

    tmp1[CubicTestData[abbreviation]["index"][PredictedCubics[abbreviation + " C"] == 1].values] = 1
    tmp1[CubicTestData[abbreviation]["index"][PredictedCubics[abbreviation + " C"] == 0].values] = 0

    tmp2[CubicTestData[abbreviation]["index"].values] = PredictedCubics[abbreviation + " C Prob"]

    Summary[abbreviation + " C"]       = tmp1.astype(int)
    Summary[abbreviation + " C Prob"]  = tmp2

In [ ]:
Summary[Summary["GB  C"]==1]

Here, we see our candidates with their probabilities.

In [ ]:
Summary[Summary["RF C"]==1]

In [ ]:
Summary.to_csv(data_dir / "summary" / "625SupervisedPredictions.csv", sep=",", index = False)